# Comparing measured and modeled data

In the previous exercise, you probably noticed that the different modeled datasets did not exactly agree on the amount of irradiance, even for the same place and time period.

The way to assess the accuracy of modeled irradiance data is to compare it with high-quality ground measurements, which is the aim of this exercise.

In [ ]:
# Install pvlib on Google Colab as this is not a standard package.
!pip install pvlib

In [6]:
import pvlib  # library for PV and solar calculations
import pandas as pd  # library for data analysis
import matplotlib.pyplot as plt  # library for plotting
import numpy as np  # library for math and linear algebra

## Step 1: Retrieve measured data

For this exercise, we will be using measurements from the Cabauw BSRN station (`station='CAB'`). The station is selected because of the high-quality, and that quality-control has already been performed on the data.

**Credentials for the BSRN FTP server are available on Learn.**.

In [ ]:
# Write your code here
# data_measured, meta_measured =  

#### SOLUTION ####
data_measured, meta_measured = pvlib.iotools.get_bsrn(
    station='CAB',
    start='2020-01-01',
    end='2020-12-31',
    username='redacted',
    password='redacted',
)

data_measured.head()

## Step 2: Resample measurement data to hourly
Since the satellite data we will be working with is hourly, we need to resample the 1-minute measurement data to hourly.

To do this, you can use the built-in function `data_measured.resample('1h').mean()`. In order for the solar position to be calculated correction for hourly data, the timestamps have to correspond to the middle of the period. Therefore, add or subtract half an hour from the index depending on whether the data is left or right-labeled. For left-labeled data, the hour 00:00 to 01:00 would be labeled 00:00.

In [ ]:
# Write your code here

#### SOLUTION ####
data_measured = data_measured.resample('1h').mean(label='left')

data_measured.index = data_measured.index + pd.Timedelta(minutes=60)

data_measured.head()

## Step 3: Calculate solar position

Calculate the solar position for the timestamps of the hourly indexes from the previous step.

Look through the exercises on "Calculating solar position in Python" if you need a refresher on how to do this.

In [ ]:
# Write your code here

#### SOLUTION ####
location = pvlib.location.Location(
    latitude=meta['latitude'],
    longitude=meta['longitude'],
)

solpos = location.get_solarposition(data_measured.index)

solpos.head()

## Step 4: Retrieve modeled data

For the comparison, we will only use one modeled dataset, namely CAMS Radiation (see the previous exercise on how to retrieve this).

Retrieve CAMS irradiance data for the same location and period:

In [ ]:
# Write your code here
data_model, meta_model = pvlib.iotools.get_cams(
    latitude=location.latitude,
    longitude=location.longitude,
    start='2020-01-01',
    end='2020-12-31 23:59',
    email='arajen@dtu.dk',
    identifier='cams_radiation',
    time_step='1h',
    label='left',
)

data_model.index = data_model.index + pd.Timedelta(minutes=30)

data_model.head()

## Step 5: Visual comparison

As a first assessment, it is always useful to plot the data.

There are many useful plots to make, but as a first inspection, a scatter plot works nicely.

What other useful plots can you think of?

In [ ]:
# Write your code here

#### SOLUTION ####
components = ['ghi', 'dhi', 'dni']

fig, axes = plt.subplots(ncols=3, figsize=(10, 4), sharey=True)
for ax, c in zip(axes, components):
    ax.set_title(c)
    ax.scatter(data_measured[c], data_model[c])

axes[0].set_ylabel('Modeled irradiance [W/m$^2$]')
axes[1].set_xlabel('Measured irradiance [W/m$^2$]')

## Step 6: Calculate deviation metrics

While the graphical comparison is a good starting point, it doesn't give any quantitative metrics.

To quantitatively compare two datasets, it is common practice to calculate the following deviation metrics:
- Root mean square deviation
- Mean absolute deviation
- Mean deviation (bias)

Calculate these deviation metrics for solar elevation angles greater than 10 degrees (to avoid using measurements that are affected by shading).

In [ ]:
# Write your code here

is_daytime = solpos['apparent_zenith'] < 90

for c in components:
    rmsd = np.sqrt(np.mean((data_measured.loc[is_daytime, c] - data_model.loc[is_daytime, c])**2))
    mad = np.mean(np.abs(data_measured.loc[is_daytime, c] - data_model.loc[is_daytime, c]))
    bias = np.mean(data_measured.loc[is_daytime, c] - data_model.loc[is_daytime, c])
    print(f"Component: {c}")
    print(f"RMSD: {rmsd:.1f}")
    print(f"MAD: {mad:.1f}")
    print(f"Bias: {bias:.1f}")

## Step 7: Compare distribution function

The deviation metrics gave high-level insight into the deviation between the datasets, but did not indicate how and why the data deviated.

To investigate this, it is useful to compare the distribution of observed irradiances between the two datasets.

In this step, plot the distribution function of GHI for the two datasets.

*Hint: Plotting overlapping histograms is one way to do this. You can make the plots semi-transparent using the `alpha` keyword.*

In [19]:
# Write your code here
fig, axes = plt.subplots(ncols=len(components), sharey=True, figsize=(10, 4))
for ax, c in zip(axes, components):
    ax.plot.hist(df_measured.loc[is_datyime, c], c='C0')
    ax.plot.hist(df_model.loc[is_datyime, c], c='C1', alpha=0.5)

axes[1].legend()